In [1]:
import sys
sys.path.append('/home/dnxx/RMNIM/')
from ip import *

In [2]:
import numpy as np
import trimesh
from ac3D.snake3D import evolution_snake3d
from ac3D.meshing import trisphere
import scipy.ndimage as ndi

In [3]:
# # Create a synthetic 3D volume (e.g., a sphere)
# size = 100
# volume = np.ones((size, size, size))
# x, y, z = np.ogrid[-size/2:size/2, -size/2:size/2, -size/2:size/2]
# mask = x**2 + y**2 + z**2 <= (size/3)**2
# volume[mask] = 1

# # Add noise and blur
# volume = ndi.gaussian_filter(volume + 0.2*np.random.randn(*volume.shape), sigma=2)

# # Create a 3D volume
# size = 100
# volume = np.zeros((size, size, size))  # background is 0

# # Define cube bounds (centered cube)
# cube_size = size // 3
# start = (size - cube_size) // 2
# end = start + cube_size

# # Create cube by setting voxel values to 1
# volume[start:end, start:end, start:end] = 1
# Create a 3D volume
size = 100
volume = np.zeros((size, size, size))  # background is 0

# Ellipsoid axes (semi-axis lengths)
a, b, c = size // 3, size // 4, size // 5  # along x, y, z

# Generate coordinate grid
x, y, z = np.ogrid[-size/2:size/2, -size/2:size/2, -size/2:size/2]

# Ellipsoid equation
mask = (x / a)**2 + (y / b)**2 + (z / c)**2 <= 1

# Fill ellipsoid
volume[mask] = 1

# Add noise and blur
volume = ndi.gaussian_filter(volume + 0.2 * np.random.randn(*volume.shape), sigma=2)

# volume = load_tif_stack("./OPF")

In [4]:
import plotly.graph_objects as go

def visualize_volume_plotly(volume, threshold=0.5):
    """Create interactive 3D visualization with Plotly showing multiple slices and isosurface"""
    # Create figure
    fig = go.Figure()
    
    # Create a meshgrid for proper coordinates
    x, y, z = np.mgrid[0:volume.shape[0], 0:volume.shape[1], 0:volume.shape[2]]
    
    # Create isosurface
    fig.add_trace(go.Isosurface(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        value=volume.flatten(),
        isomin=threshold,
        isomax=1.0,
        opacity=0.3,
        surface_count=1,
        colorscale='Viridis',
        caps=dict(x_show=False, y_show=False, z_show=False)
    ))
    
    # Get middle slice indices
    mid_x = volume.shape[0] // 2
    mid_y = volume.shape[1] // 2
    mid_z = volume.shape[2] // 2
    
    # Add X-axis slice (YZ plane)
    fig.add_trace(go.Surface(
        z=np.ones((volume.shape[1], volume.shape[2])) * mid_x,
        x=np.ones((volume.shape[1], volume.shape[2])) * mid_x,
        y=np.mgrid[0:volume.shape[1], 0:volume.shape[2]][0],
        surfacecolor=volume[mid_x],
        colorscale='Viridis',
        opacity=0.7,
        showscale=False
    ))
    
    # Add Y-axis slice (XZ plane)
    fig.add_trace(go.Surface(
        z=np.mgrid[0:volume.shape[0], 0:volume.shape[2]][0],
        x=np.mgrid[0:volume.shape[0], 0:volume.shape[2]][0],
        y=np.ones((volume.shape[0], volume.shape[2])) * mid_y,
        surfacecolor=volume[:, mid_y, :],
        colorscale='Viridis',
        opacity=0.7,
        showscale=False
    ))
    
    # Add Z-axis slice (XY plane)
    fig.add_trace(go.Surface(
        z=np.ones((volume.shape[0], volume.shape[1])) * mid_z,
        x=np.mgrid[0:volume.shape[0], 0:volume.shape[1]][0],
        y=np.mgrid[0:volume.shape[0], 0:volume.shape[1]][1],
        surfacecolor=volume[:, :, mid_z],
        colorscale='Viridis',
        opacity=0.7,
        showscale=False
    ))
    
    # Layout settings
    fig.update_layout(
        title="3D Volume Visualization",
        width=800,
        height=800,
        scene=dict(
            xaxis=dict(title="X"),
            yaxis=dict(title="Y"),
            zaxis=dict(title="Z"),
            aspectratio=dict(x=1, y=1, z=1)
        )
    )
    
    return fig

In [5]:
# # Example usage
# fig = visualize_volume_plotly(volume, threshold=0.1)
# fig.show()

In [6]:
# Create initial mesh (slightly larger than the object)
vertices, faces = trisphere(iterations=5, radius=40, x_0=size//2, y_0=size//2, z_0=size//2)
mesh_init = trimesh.Trimesh(vertices=vertices, faces=faces)

In [7]:
mesh_init.show()

In [8]:
# Run snake evolution
final_mesh = evolution_snake3d(
    volume, 
    mesh_init,
    alpha=300,
    beta=50,
    gamma=0.1,
    kb=10,  # Small balloon force (shrinking)
    max_iter=1000,
    verbose=False
)

In [9]:
# Visualize the result
final_mesh.show()